# Uninformed and Informed Search Agent
- Start with a frontier that contains the initial state (**node**, actually).
- Start with an empty explored **set**.
- Repeat:
    - If the frontier **is empty**, then no solution.
    - **Remove a node** from the frontier.
    - If node contains **goal state**, return the solution.
    - Add the node to the **explored set**.
    - **Expand node**, add resulting nodes to the frontier if they aren't already in the frontier or the explored set.
    - Use the **cost**
    

In [1]:
class Node:
    """ The Node class is used in the search algorithms.
    It can be trivially extended to include costs
    """
    def __init__(self, state, parent=None, action=None, cost=0):
        """ The constructor needs the current state and action, and a pointer to a
        parent Node.
        For the 'root' node, only the start state.
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        
    def __str__(self):
        """ Returns a string with the state and action of this node"""
        return f"State: {self.state} Action: {self.action} Cost: {self.cost}\n"


![](graph.png)

In [2]:
adjacency = ([
[0,2,1,0,0,0,3,0], # S
[0,4,0,2,3,0,0,0], # A
[0,0,5,0,2,4,0,0], # B
[0,0,0,2,0,0,0,4], # C
[0,0,0,0,2,0,0,4], # D
[0,0,0,0,0,8,0,0], # E
[0,0,0,0,0,0,4,6], # F
[0,0,0,0,0,0,0,0] # G
],  
0, # Start state
7,# Goal state
["S","A","B","C","D","E","F","G"]) # Labels

In [3]:
class Environment(object):
    """ This class is model specific (not universal at all!)"""
    def __init__(self, adjacency):
        self.adj = adjacency[0] # List of "walls" encoded with "ones"
        self.start = adjacency[1] # tuple with initial coordinates
        self.goal = adjacency[2] # tuple with goal coordinates
        self.names = adjacency[3]
        
    def print(self,solution=None): # Some fancy printing. If solution is provided, it's fancier :-D
        for row in self.adj:
            print(row)
        print(f"Start:{self.start}")
        print(f"Goal: {self.goal}")
        print(f"Heuristic: {[adjacency[0][i][i] for i in range(len(adjacency[0][0]))]}")


In [4]:
graph_environment = Environment(adjacency)
graph_environment.print()

[0, 2, 1, 0, 0, 0, 3, 0]
[0, 4, 0, 2, 3, 0, 0, 0]
[0, 0, 5, 0, 2, 4, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 4]
[0, 0, 0, 0, 2, 0, 0, 4]
[0, 0, 0, 0, 0, 8, 0, 0]
[0, 0, 0, 0, 0, 0, 4, 6]
[0, 0, 0, 0, 0, 0, 0, 0]
Start:0
Goal: 7
Heuristic: [0, 4, 5, 2, 2, 8, 4, 0]


In [5]:
class SearchAgent:
    """ Generic class for search algorithms"""
    def __init__(self,environment):
        """ The constructor requires an environment class that, at least contains
        the start and goal states.
        
        If additional information is required, this class can be inherited but you
        should include:
        super().__init__()
        """
        self.frontier = [Node(state=environment.start)] # We play with nodes, not states.
        self.explored = set() # I use a 'set' to avoid repetitions (and order doesn't matter)
        self.goal_state = environment.goal
    
    def print_frontier(self): 
        """ Print the nodes in the frontier"""
        print("Frontier:")
        print("---------")
        for node in self.frontier:
            print(node)
            
    def print_explored(self):
        """ Print the nodes in the explored set"""
        print("Explored:")
        print("---------")
        for node in self.explored:
            print(node)
            
    def is_frontier_empty(self):
        """ A boolean. True if the frontier is empty (so there is no solution)"""
        return len(self.frontier) == 0 # No solution

    def actions(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def result(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def remove_node(self):
        """ Needs to be extended in the child daughter classes of this class.
        Here is where we decide if we use a stack, a queue, ...
        """
        pass
    
    def is_goal(self,node):
        """ This function plays two roles:
        1. Check if we have currently reached the goal state.
        2. If so, reconstruct the tree backwards to show the obtained solution.
        """
        if node.state == self.goal_state:
            self.solution = []
            while node.parent is not None:
                self.solution.append(node)
                print(f"{node.state}({node.action})")
                node = node.parent
            raise Exception("Solution found")

    def add_to_frontier(self,node):
        """ Append node to the frontier. """
        self.frontier.append(node)
            
    def add_to_explored(self,node):
        """Add node to explored set (this is a pythonic "set" that's why I use add instead of append."""
        self.explored.add(node) 

    def is_state_in_frontier(self,node):
        """Returns a boolean (True) if the state is in the current frontier"""
        return any(frontier_node.state == node.state for frontier_node in self.frontier)
  
    def is_state_in_explored(self,node):
        """Returns a boolean (True) if the state is in the explored set"""
        return any(explored_node.state == node.state for explored_node in self.explored)
    
    def expand_node(self, node):
        """Here is the main function of the code. We decide RESULTs taking the states and 
        actions into consideration.
        
        I've tried to make this code as versatile as I can. This part should be the same
        for any search problem you program. Cool, uh!
        """
        for child in self.result(node): 
            # Check if the node is not frontier or explored
            if not self.is_state_in_frontier(child) and not self.is_state_in_explored(child):
                self.add_to_frontier(child) # add all the "compatible" children
        
    def solve(self,verbose=0):
        """The main loop. Here is where we implement the "Repeat" part of the algorithm. 
        It follows almost verbatim the description given in the lecture"""
        while not self.is_frontier_empty():
            if verbose!=0:
                self.print_frontier()
                self.print_explored()
            try:
                node = self.remove_node()
            except:
                print("Solution not found!!")
                
            try:
                self.is_goal(node)
            except:
                return self.solution
            self.add_to_explored(node)
            self.expand_node(node)

In [6]:
class AgentDepthFirst(SearchAgent):
    def remove_node(self):
        node = self.frontier.pop()
        print(f"Extracting node: {node}")
        return node # Remove the last element (this is a stack!!!)
    

class GraphDFSearch(AgentDepthFirst):
    def __init__(self,environment):
        super().__init__(environment) # Inherit constructor from parent
        self.adj = environment.adj # Store walls to be used in function RESULTS(s,a) below
        self.names = environment.names
        
    def actions(self,node):
        s = node.state
        n = len(self.adj)
        possible_actions = [(f"go to {self.names[i]}",i) for i in range(n) if self.adj[s][i]!=0 and i!=s]
        
        print(possible_actions)
        return possible_actions
        
    def result(self,node):
        children = []
        
        for action, s in self.actions(node):
            children.append(Node(state=s,parent=node,action=action))
                
        return children
        

In [7]:
graph_environment = Environment(adjacency)
graph_environment.print()

graph_solver = GraphDFSearch(graph_environment)
solution_nodes = graph_solver.solve(1)


[0, 2, 1, 0, 0, 0, 3, 0]
[0, 4, 0, 2, 3, 0, 0, 0]
[0, 0, 5, 0, 2, 4, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 4]
[0, 0, 0, 0, 2, 0, 0, 4]
[0, 0, 0, 0, 0, 8, 0, 0]
[0, 0, 0, 0, 0, 0, 4, 6]
[0, 0, 0, 0, 0, 0, 0, 0]
Start:0
Goal: 7
Heuristic: [0, 4, 5, 2, 2, 8, 4, 0]
Frontier:
---------
State: 0 Action: None Cost: 0

Explored:
---------
Extracting node: State: 0 Action: None Cost: 0

[('go to A', 1), ('go to B', 2), ('go to F', 6)]
Frontier:
---------
State: 1 Action: go to A Cost: 0

State: 2 Action: go to B Cost: 0

State: 6 Action: go to F Cost: 0

Explored:
---------
State: 0 Action: None Cost: 0

Extracting node: State: 6 Action: go to F Cost: 0

[('go to G', 7)]
Frontier:
---------
State: 1 Action: go to A Cost: 0

State: 2 Action: go to B Cost: 0

State: 7 Action: go to G Cost: 0

Explored:
---------
State: 6 Action: go to F Cost: 0

State: 0 Action: None Cost: 0

Extracting node: State: 7 Action: go to G Cost: 0

7(go to G)
6(go to F)


In [8]:
class GraphBFSearch(GraphDFSearch):
    def remove_node(self):
        node = self.frontier.pop(0)
        print(f"Extracting node: {node}")
        return node # Remove the first element (this is a queue!)

In [9]:
graph_solverB = GraphBFSearch(graph_environment)
solution_nodesB = graph_solverB.solve(1)

Frontier:
---------
State: 0 Action: None Cost: 0

Explored:
---------
Extracting node: State: 0 Action: None Cost: 0

[('go to A', 1), ('go to B', 2), ('go to F', 6)]
Frontier:
---------
State: 1 Action: go to A Cost: 0

State: 2 Action: go to B Cost: 0

State: 6 Action: go to F Cost: 0

Explored:
---------
State: 0 Action: None Cost: 0

Extracting node: State: 1 Action: go to A Cost: 0

[('go to C', 3), ('go to D', 4)]
Frontier:
---------
State: 2 Action: go to B Cost: 0

State: 6 Action: go to F Cost: 0

State: 3 Action: go to C Cost: 0

State: 4 Action: go to D Cost: 0

Explored:
---------
State: 0 Action: None Cost: 0

State: 1 Action: go to A Cost: 0

Extracting node: State: 2 Action: go to B Cost: 0

[('go to D', 4), ('go to E', 5)]
Frontier:
---------
State: 6 Action: go to F Cost: 0

State: 3 Action: go to C Cost: 0

State: 4 Action: go to D Cost: 0

State: 5 Action: go to E Cost: 0

Explored:
---------
State: 0 Action: None Cost: 0

State: 2 Action: go to B Cost: 0

State: 1 